In [1]:
# SAFE TO CSV
import pandas as pd
from typing import List

def parse_csv(path, col_idxs:List[int]=None):
    assert path.endswith(".csv"), f"{path} is not a valid .csv file"
    dset = pd.read_csv(path)
    names = list(dset.keys())
    out = {}
    for col_i in col_idxs:
       out[names[col_i]] = dset[names[col_i]].to_list()

    # Assert that all values in the dictionary have the same length
    assert all(len(v) == len(next(iter(out.values()))) for v in out.values()), "All dictionary values must have the same length"
    return out

In [ ]:
path:str = '/home/nobilm@usi.ch/pretrain_paper/data/moelculenet/HIV.csv'
out = parse_csv(path, [0,2]) # bace: [0,2], freesolv: [1,2], esol [1, -1], lipo [1,2],
smiles = out['smiles'] # bace: mol, freesolv: smiles, esol: smiles, lipo: smiles
ys = out['HIV_active'] # bace: Class, freesolv: expt ,esol 'ESOL predicted log solubility in mols per litre', lipo: exp, bbbp:p_np
assert len(smiles) == len(ys)
len(smiles)

41127

In [5]:
import safe

num_of_chunks_in_mol = []
safe_smi_list = []
for smi, y in zip(smiles, ys):
    try:
        safe_smi = safe.encode(smi)
        safe_smi_list.append(safe_smi)
    except:
        num_of_chunks_in_mol.append(1)
        safe_smi_list.append(smi)
        continue
    if '.' in safe_smi:
        num_of_chunks_in_mol.append(len(safe_smi.split('.')))
    else:
        num_of_chunks_in_mol.append(1)

assert len(smiles) == len(ys) == len(num_of_chunks_in_mol) == len(safe_smi_list)

/home/nobilm@usi.ch/miniconda3/envs/safe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import csv

# Define the header
header = ['smiles', 'safe_smi', 'num_of_chunks_in_mol', 'ys']

# Open a new CSV file for writing
with open('/home/nobilm@usi.ch/pretrain_paper/data/moelculenet/SAFEHIV.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header
    writer.writerow(header)

    # Write the data
    for row in zip(smiles, safe_smi_list, num_of_chunks_in_mol, ys):
        writer.writerow(row)

# without y

In [2]:
path:str = '/home/nobilm@usi.ch/pretrain_paper/data/zinc15_250K.csv'
out = parse_csv(path, [0]) # bace: [0,2], freesolv: [1,2], esol [1, -1], lipo [1,2],
smiles = out['smiles']

In [3]:
import safe

num_of_chunks_in_mol = []
safe_smi_list = []
for smi in smiles:
    try:
        safe_smi = safe.encode(smi)
        safe_smi_list.append(safe_smi)
    except:
        num_of_chunks_in_mol.append(1)
        safe_smi_list.append(smi)
        continue
    if '.' in safe_smi:
        num_of_chunks_in_mol.append(len(safe_smi.split('.')))
    else:
        num_of_chunks_in_mol.append(1)

assert len(smiles) == len(num_of_chunks_in_mol) == len(safe_smi_list)

/home/nobilm@usi.ch/miniconda3/envs/safe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-19 19:34:18.012 | WARNING  | safe.converter:encoder:363 - Ignoring stereo is disabled, but molecule has stereochemistry interferring with SAFE representation
2025-05-19 19:34:18.046 | WARNING  | safe.converter:encoder:363 - Ignoring stereo is disabled, but molecule has stereochemistry interferring with SAFE representation
2025-05-19 19:34:18.164 | WARNING  | safe.converter:encoder:363 - Ignoring stereo is disabled, but molecule has stereochemistry interferring with SAFE representation
2025-05-19 19:34:18.270 | WARNING  | safe.converter:encoder:363 - Ignoring stereo is disabled, but molecule has stereochemistry interferring with SAFE representation
2025-05-19 19:34:18.633 | WARNING  | safe.converter:encoder:3

In [4]:
import numpy as np
np.min(num_of_chunks_in_mol), np.max(num_of_chunks_in_mol), np.mean(num_of_chunks_in_mol), np.std(num_of_chunks_in_mol), np.median(num_of_chunks_in_mol), len(np.bincount(num_of_chunks_in_mol)), np.bincount(num_of_chunks_in_mol)

(np.int64(1),
 np.int64(12),
 np.float64(5.985892),
 np.float64(1.5306433171500144),
 np.float64(6.0),
 13,
 array([    0,   277,  2320, 10808, 28248, 49770, 64706, 55247, 27654,
         8867,  1860,   226,    17]))

In [5]:
import csv

# Define the header
header = ['smiles', 'safe_smi', 'num_of_chunks_in_mol']

# Open a new CSV file for writing
with open('/home/nobilm@usi.ch/pretrain_paper/data/zinc_with_safe.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header
    writer.writerow(header)

    # Write the data
    for row in zip(smiles, safe_smi_list, num_of_chunks_in_mol):
        writer.writerow(row)

In [6]:
with open('/home/nobilm@usi.ch/pretrain_paper/data/zinc_with_safe.csv', 'r') as f:
    line_count = sum(1 for _ in f)
line_count

250001

# SAFE EXPLORATION

In [ ]:
from rdkit import Chem as rdChem
def plot_mol_with_atom_idxs(mol):
    from rdkit.Chem.Draw import IPythonConsole
    IPythonConsole.drawOptions.addAtomIndices = True
    return mol

In [ ]:
import safe

ibuprofen = "CC(Cc1ccc(cc1)C(C(=O)O)C)C"
ibuprofen2 = 'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O'

# 2 different smiles get mapped to same safe repr

ibuprofen_sf = safe.encode(ibuprofen)  # c12ccc3cc1.C3(C)C(=O)O.CC(C)C2
ibuprofen_sf2 = safe.encode(ibuprofen2)  # c12ccc3cc1.C3(C)C(=O)O.CC(C)C2
ibuprofen_sf, ibuprofen_sf2, ibuprofen_sf == ibuprofen_sf2

In [ ]:
safe.encode(ibuprofen2)

In [ ]:
mol2 = rdChem.MolFromSmiles(ibuprofen_sf2)
mol2

In [ ]:
mol = rdChem.MolFromSmiles(ibuprofen_sf)
mol

In [ ]:
for atom in mol.GetAtoms():
    print(atom.GetIdx(), atom.GetSymbol())

In [ ]:
plot_mol_with_atom_idxs(mol)

In [ ]:
suppl = rdChem.SDMolSupplier('/storage_common/nobilm/pretrain_paper/PCQM4Mv2Dataset/from_wget/pcqm4m-v2-train.sdf')
len(suppl)

In [ ]:
mols_test = []
for idx, mol in enumerate(suppl):
    if not mol:continue
    if idx == 10000:
        break
    mols_test.append(mol)



In [ ]:
fragments = set()
for idx, mol in enumerate(suppl):
    if idx == 10000:
        break
    if mol:
        smi = rdChem.MolToSmiles(mol)
    else: continue
    try:
        smi_as_safe = safe.encode(smi)
        if '.' in smi_as_safe:
            for el in smi_as_safe.split('.'):
                fragments.add(el)
        else:
            fragments.add(smi_as_safe)
    except:
        pass

In [ ]:
len(fragments)

In [ ]:
list_f = []
for f in fragments:
    m = rdChem.MolFromSmiles(f)
    list_f.append(m)

In [ ]:
list_f

In [ ]:
# Richard hall 2017
# IFG main code
# Guillaume Godin 2017
# refine output function
# astex_ifg: identify functional groups a la Ertl, J. Cheminform (2017) 9:36
from rdkit import Chem
from collections import namedtuple

def merge(mol, marked, aset):
    bset = set()
    for idx in aset:
        atom = mol.GetAtomWithIdx(idx)
        for nbr in atom.GetNeighbors():
            jdx = nbr.GetIdx()
            if jdx in marked:
                marked.remove(jdx)
                bset.add(jdx)
    if not bset:
        return
    merge(mol, marked, bset)
    aset.update(bset)

# atoms connected by non-aromatic double or triple bond to any heteroatom
# c=O should not match (see fig1, box 15).  I think using A instead of * should sort that out?
PATT_DOUBLE_TRIPLE = Chem.MolFromSmarts('A=,#[!#6]')
# atoms in non aromatic carbon-carbon double or triple bonds
PATT_CC_DOUBLE_TRIPLE = Chem.MolFromSmarts('C=,#C')
# acetal carbons, i.e. sp3 carbons connected to tow or more oxygens, nitrogens or sulfurs; these O, N or S atoms must have only single bonds
PATT_ACETAL = Chem.MolFromSmarts('[CX4](-[O,N,S])-[O,N,S]')
# all atoms in oxirane, aziridine and thiirane rings
PATT_OXIRANE_ETC = Chem.MolFromSmarts('[O,N,S]1CC1')

PATT_TUPLE = (PATT_DOUBLE_TRIPLE, PATT_CC_DOUBLE_TRIPLE, PATT_ACETAL, PATT_OXIRANE_ETC)

def identify_functional_groups(mol):
    marked = set()
#mark all heteroatoms in a molecule, including halogens
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum() not in (6,1): # would we ever have hydrogen?
            marked.add(atom.GetIdx())

#mark the four specific types of carbon atom
    for patt in PATT_TUPLE:
        for path in mol.GetSubstructMatches(patt):
            for atomindex in path:
                marked.add(atomindex)

#merge all connected marked atoms to a single FG
    groups = []
    while marked:
        grp = set([marked.pop()])
        merge(mol, marked, grp)
        groups.append(grp)

#extract also connected unmarked carbon atoms
    ifg = namedtuple('IFG', ['atomIds', 'atoms', 'type'])
    ifgs = []
    for g in groups:
        uca = set()
        for atomidx in g:
            for n in mol.GetAtomWithIdx(atomidx).GetNeighbors():
                if n.GetAtomicNum() == 6:
                    uca.add(n.GetIdx())
        ifgs.append(ifg(atomIds=tuple(list(g)), atoms=Chem.MolFragmentToSmiles(mol, g, canonical=True), type=Chem.MolFragmentToSmiles(mol, g.union(uca),canonical=True)))
    return ifgs


# for ix, smiles in enumerate([
#     # 'Cc1nc(NS(=O)(=O)c2ccc(N)cc2)nc(C)c1', # fig1, 1
#     # 'NC(=N)c1ccc(C=Cc2ccc(cc2O)C(=N)N)cc1', # 2
#     # 'CC(=O)Nc1nnc(s1)S(=O)(=O)N', # 3
#     # 'NS(=O)(=O)c1cc2c(NCNS2(=O)=O)cc1Cl', # 4
#     # 'CNC1=Nc2ccc(Cl)cc2C(=N(=O)C1)c3ccccc3', # 5
#     # 'Cc1onc(c1C(=O)NC2C3SC(C)(C)C(N3C2=O)C(=O)O)c4ccccc4', # 6
#     # 'Clc1ccccc1C2=NCC(=O)Nc3ccc(cc23)N(=O)=O', # 7
#     # 'COc1cc(cc(C(=O)NCC2CCCN2CC=C)c1OC)S(=O)(=O)N', # 8
#     # 'Cc1ccc(Cl)c(Nc2ccccc2C(=O)O)c1Cl', # 9
#     # 'Clc1ccc2Oc3ccccc3N=C(N4CCNCC4)c2c1', # 10 - there is a discrepancy with the paper here!  I wonder if Peter has the ring as aromatic?
#     # 'FC(F)(F)CN1C(=O)CN=C(c2ccccc2)c3cc(Cl)ccc13', # 11
#     # 'OCC1OC(CC1O)n2cnc3C(O)CNC=Nc32', # 12
#     # 'CCNC1CC(C)S(=O)(=O)c2sc(cc12)S(=O)(=O)N', # 13
#     # 'CC(O)C1C2C(C)C(=C(N2C1=O)C(=O)O)SC3CNC(C3)C(=O)N(C)C', # 14
#     # 'CC1CN(CC(C)N1)c2c(F)c(N)c3c(=O)c(cn(C4CC4)c3c2F)C(=O)O', # 15
#     # 'CC(=CCC1C(=O)N(N(C1=O)c2ccccc2)c3ccccc3)C', # 16
#     # 'Clc1ccc2N=C3NC(=O)CN3Cc2c1Cl', # 17
#     # 'CC(=O)NC1C(NC(=N)N)C=C(OC1C(O)C(O)CO)C(=O)O', # 18
#     # 'CC(O)C(O)C1CNc2nc(N)nc(O)c2N1', # 19
#     # 'NC1CCCCN(C1)c2c(Cl)cc3c(=O)c(cn(C4CC4)c3c2Cl)C(=O)O', # 20
#     'CCO',
#     'CC(=O)C',
#     'CC(=O)O', #
#     'c1ccccc1',
#     'CC(=O)NC',# N
#     'CCN',
#     'CC#N',
#     'CC(=O)Oc1ccccc1C(=O)O',
#     'CCO',
#     'c1ccccc1',
#     'C[N+](=O)[O-]',
#     'CS(=O)(=O)C',
#     'c1ccccc1Br',
# ]):
#     m = Chem.MolFromSmiles(smiles)
#     fgs = identify_functional_groups(m)
#     print('%2d: %d fgs'%(ix, len(fgs)), fgs)




In [ ]:
types = set()
for ix , m in enumerate(mols_test):
    fgs = identify_functional_groups(m)
    for el in fgs:
        types.add(el.type)

In [ ]:
len(types)

In [ ]:
types

In [ ]:
mmmm = []
for t in types:
    mmmm.append(rdChem.MolFromSmiles(t))


In [ ]:
len(mmmm)


In [ ]:
casted = []
for m in mmmm:
    print(m)
    if m is None:continue
    casted.append(m)



In [ ]:
len(casted)


In [ ]:
len(set(casted))

In [ ]:
casted[0]

In [ ]:
from copy import deepcopy
asd = deepcopy(casted[0] )

In [ ]:
print(Chem.MolToSmiles(casted[0]) == Chem.MolToSmiles(asd) )
print(Chem.MolToInchi(casted[0]) == Chem.MolToInchi(asd) )

In [ ]:
uniques = []
skip_idxs = []
for idx_i, m in enumerate(casted):
    duplicate = False

    if idx_i in skip_idxs:
        continue
    for idx_j in range(idx_i+1, len(casted)):
        mm = casted[idx_j]
        if Chem.MolToInchi(m) == Chem.MolToInchi(mm):
            skip_idxs.append(idx_j)
            duplicate = True
    if not duplicate:
        uniques.append(idx_i)


In [ ]:
with open("asdasd.txt", "w") as log_file:
    log_file.write(f"Number of unique molecules: {len(uniques)}\n")
    log_file.write(f"Number of casted molecules: {len(casted)}\n")